In [5]:
%config Completer.use_jedi = False

In [1]:
import os
import sys
RunPath = os.getcwd() + "/"
CrysDatPath = "/home/sohamc2/HEA_FCC/CrysDat/"
DataPath = "/home/sohamc2/HEA_FCC/MDMC/ML_runs/DataSets/"
ModulePath = "/home/sohamc2/VKMC/SymNetworkRuns/CE_Symmetry/Symm_Network/"

In [2]:
import numpy as np
import h5py
import torch as pt
from tqdm import tqdm
import pickle
from GCNetRun import Load_Data, makeComputeData, makeProdTensor, Load_crysDats

In [4]:
T = 1073
state1List, state2List, dispList, rateList, AllJumpRates, jmpSelects = Load_Data("singleStep_{}.h5".format(T))

found permuation


In [6]:
GpermNNIdx, NNsiteList, siteShellIndices, GIndtoGDict, JumpNewSites, dxJumps = Load_crysDats()
RtoSiteInd = np.load(CrysDatPath + "RtoSiteInd.npy")
SiteIndToR = np.load(CrysDatPath + "SiteIndtoR.npy")

with open(CrysDatPath + "supercellFCC.pkl", "rb") as fl:
    superFCC = pickle.load(fl)
N_units = superFCC.superlatt[0,0]
N_units

8

# first test with single step dataset

In [7]:
specCheck = 5
specsToTrain = [specCheck]
VacSpec = 0
N_train = 2000
AllJumps = False
State1_occs, State2_occs, rateData, dispData, OnSites_state1, OnSites_state2 =\
makeComputeData(state1List, state2List, dispList, specsToTrain,
                VacSpec, rateList, AllJumpRates, JumpNewSites,
                dxJumps, NNsiteList, N_train, AllJumps=AllJumps)

OnSiteTensor1 = makeProdTensor(OnSites_state1, 3)
OnSiteTensor2 = makeProdTensor(OnSites_state2, 3)

  1%|          | 22/4000 [00:00<00:18, 212.77it/s]

Building Occupancy Tensors for species : [5]


100%|██████████| 4000/4000 [00:20<00:00, 199.10it/s]


In [9]:
for samp in tqdm(range(1000), position=0, leave=True):
    for site in range(state1List.shape[1]):
        if site == 0:
            assert np.all(State1_occs[samp, :, site] == 0)
        else:
            spec1 = state1List[samp, site]
            assert State1_occs[samp, spec1 - 1, site] == 1
            assert np.sum(State1_occs[samp, :, site]) == 1
            
            spec2 = state2List[samp, site]
            assert State2_occs[samp, spec2 - 1, site] == 1
            assert np.sum(State2_occs[samp, :, site]) == 1
        
    # check the displacements
    jSelect = jmpSelects[samp]
    NNR = np.dot(np.linalg.inv(superFCC.crys.lattice), dxJumps[jSelect]).astype(int) % N_units
    NNsiteVac = NNsiteList[jSelect + 1, 0]
    assert np.all(NNR == SiteIndToR[NNsiteVac])
    spec = state1List[samp, NNsiteVac]
    assert np.allclose(dispData[samp, 0], 3.59*dxJumps[jSelect])
    if spec == specCheck:
        assert np.allclose(dispData[samp, 1], -3.59*dxJumps[jSelect])
    else:
        assert np.allclose(dispData[samp, 1], 0.0)
    
    # check the rate
    assert np.math.isclose(rateData[samp], rateList[samp])

100%|██████████| 1000/1000 [00:36<00:00, 27.66it/s]


In [10]:
OnSiteTensor1.shape

torch.Size([4000, 3, 512])

In [11]:
batchStart = np.random.randint(OnSiteTensor1.shape[0] - 1000)
batchEnd = batchStart + 1000
y = pt.rand(1000, 3, state1List.shape[1]).double()

y1_calc = pt.sum(y*OnSiteTensor1[batchStart : batchEnd], dim=2)
y2_calc = pt.sum(y*OnSiteTensor2[batchStart : batchEnd], dim=2)

for sampInd in tqdm(range(batchStart, batchEnd), position=0, leave=True):
    y1_samp = pt.zeros(3).double()
    y2_samp = pt.zeros(3).double()
    for site in range(OnSiteTensor1.shape[2]):
        occs1 = State1_occs[sampInd, :, site]
        occs2 = State2_occs[sampInd, :, site]
        if occs1[specCheck-1] == 1:
            y1_samp += y[sampInd - batchStart, :, site]
        if occs2[specCheck-1] == 1:
            y2_samp += y[sampInd - batchStart, :, site]
    
    assert pt.allclose(y1_calc[sampInd - batchStart], y1_samp)
    assert pt.allclose(y2_calc[sampInd - batchStart], y2_samp)

100%|██████████| 1000/1000 [00:08<00:00, 117.07it/s]


# now let's try with all jumps

In [12]:
specCheck = 5
specsToTrain = [specCheck]
VacSpec = 0
N_train = 100
AllJumps = True
State1_occs, State2_occs, rateData, dispData, OnSites_state1, OnSites_state2 =\
makeComputeData(state1List, state2List, dispList, specsToTrain,
                VacSpec, rateList, AllJumpRates, JumpNewSites,
                dxJumps, NNsiteList, N_train, AllJumps=AllJumps)

assert State1_occs.shape[0] == State2_occs.shape[0] == rateData.shape[0] == dispData.shape[0] == 2400
assert OnSites_state1.shape[0] == OnSites_state1.shape[0] == 2400

OnSiteTensor1 = makeProdTensor(OnSites_state1, 3)
OnSiteTensor2 = makeProdTensor(OnSites_state2, 3)

  2%|▏         | 4/200 [00:00<00:11, 16.97it/s]

Building Occupancy Tensors for species : [5]


100%|██████████| 200/200 [00:11<00:00, 16.81it/s]


In [13]:
for stateInd in tqdm(range(N_train*2), position=0, leave=True):
    state1 = state1List[stateInd]
    for jInd in range(dxJumps.shape[0]):
        state2 = state1.copy()
        NNsiteVac = NNsiteList[jInd + 1, 0]
        RsiteVac = SiteIndToR[NNsiteVac]
        state2[0] = state2[NNsiteVac]
        state2[NNsiteVac] = 0
        
        for site in range(state1List.shape[1]):
            Rsite = SiteIndToR[site]
            RsiteNew = (Rsite + RsiteVac) % N_units
            siteNew = RtoSiteInd[RsiteNew[0], RsiteNew[1], RsiteNew[2]]
            spec1 = state1[site]
            spec2 = state2[siteNew]
            
            if site == 0:
                assert spec2 == 0
                assert np.all(State1_occs[stateInd * dxJumps.shape[0] + jInd, :, site] == 0)
                assert np.all(State2_occs[stateInd * dxJumps.shape[0] + jInd, :, site] == 0)
            
            else:
                assert spec2 != 0
                assert State1_occs[stateInd * dxJumps.shape[0] + jInd, spec1-1, site] == 1
                assert np.sum(State1_occs[stateInd * dxJumps.shape[0] + jInd, :, site]) == 1

                assert State2_occs[stateInd * dxJumps.shape[0] + jInd, spec2-1, site] == 1
                assert np.sum(State2_occs[stateInd * dxJumps.shape[0] + jInd, :, site]) == 1
        
        # check the displacements
        NNR = np.dot(np.linalg.inv(superFCC.crys.lattice), dxJumps[jInd]).astype(int) % N_units
        NNsiteVac = NNsiteList[jInd + 1, 0]
        assert np.all(NNR == SiteIndToR[NNsiteVac])
        spec = state1[NNsiteVac]
        assert np.allclose(dispData[stateInd * dxJumps.shape[0] + jInd, 0], 3.59*dxJumps[jInd])
        if spec == specCheck:
            assert np.allclose(dispData[stateInd * dxJumps.shape[0] + jInd, 1], -3.59*dxJumps[jInd])
        
        # check the rate
        assert np.allclose(rateData[stateInd * dxJumps.shape[0] + jInd], AllJumpRates[stateInd, jInd])

100%|██████████| 200/200 [01:35<00:00,  2.10it/s]


In [14]:
batchStart = np.random.randint(OnSiteTensor1.shape[0] - 1000)
batchEnd = batchStart + 1000
y = pt.rand(1000, 3, state1List.shape[1]).double()

y1_calc = pt.sum(y*OnSiteTensor1[batchStart : batchEnd], dim=2)
y2_calc = pt.sum(y*OnSiteTensor2[batchStart : batchEnd], dim=2)

for sampInd in tqdm(range(batchStart, batchEnd), position=0, leave=True):
    y1_samp = pt.zeros(3).double()
    y2_samp = pt.zeros(3).double()
    for site in range(OnSiteTensor1.shape[2]):
        occs1 = State1_occs[sampInd, :, site]
        occs2 = State2_occs[sampInd, :, site]
        if occs1[specCheck-1] == 1:
            y1_samp += y[sampInd - batchStart, :, site]
        if occs2[specCheck-1] == 1:
            y2_samp += y[sampInd - batchStart, :, site]
    
    assert pt.allclose(y1_calc[sampInd - batchStart], y1_samp)
    assert pt.allclose(y2_calc[sampInd - batchStart], y2_samp)

100%|██████████| 1000/1000 [00:08<00:00, 116.60it/s]
